## LSTM Recurrent Network for Text Generation

### LIBRARY IMPORTS

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

### LOAD DATASET AND CREATE MAP OF UNIQUE CHARACTERS

In [2]:
# Load the text corpus
filename = "Cat_in_the_Hat.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()

# Convert all the text to lowercase
raw_text = raw_text.lower()

# Create a map that maps each unique character in the text to a unique integer value
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# Display the total number of characters (n_chars) and the vocabulary (the number of unique characters)
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  7353
Total Vocab:  32


### CREATE TRAINING PATTERNS

In [3]:
# Create the patterns to be used for training
seq_length = 100 # fixed length sliding window for training pattern
dataX = [] # input sequences
dataY = [] # outputs

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  7253


### TRANSFORM DATA TO BE SUITABLE FOR KERAS 

In [4]:
# Reshape dataX to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Rescale integers mapped to characters to the range 0-to-1 to accommodate learning using sigmoid function
X = X / float(n_vocab)

# One hot encode the output variable
y = np_utils.to_categorical(dataY)

### BUILD THE LSTM MODEL

In [5]:
# Build sequential model containing 1 LSTM layers and 1 Dropout layers, followed by a dense output layer
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# Compile the model using the Adam optimizer and categorical crossentropy for the loss function
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Define the checkpoint; i.e., saved past state of the model
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### TRAINING

In [6]:
# Train the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
114/114 [==============================] - ETA: 0s - loss: 3.0351
Epoch 00001: loss improved from inf to 3.03508, saving model to weights-improvement-01-3.0351-bigger.hdf5
114/114 [==============================] - 18s 145ms/step - loss: 3.0351
Epoch 2/50
114/114 [==============================] - ETA: 0s - loss: 2.9865
Epoch 00002: loss improved from 3.03508 to 2.98648, saving model to weights-improvement-02-2.9865-bigger.hdf5
114/114 [==============================] - 17s 150ms/step - loss: 2.9865
Epoch 3/50
114/114 [==============================] - ETA: 0s - loss: 2.9755
Epoch 00003: loss improved from 2.98648 to 2.97550, saving model to weights-improvement-03-2.9755-bigger.hdf5
114/114 [==============================] - 25s 218ms/step - loss: 2.9755
Epoch 4/50
114/114 [==============================] - ETA: 0s - loss: 2.9449
Epoch 00004: loss improved from 2.97550 to 2.94491, saving model to weights-improvement-04-2.9449-bigger.hdf5
114/114 [============================